In [1]:
from model import ModelConfig, LlamaModel
from train import TrainerConfig, DataLoader, Trainer

from transformers import AutoTokenizer

In [2]:
tokenizer_id = "HuggingFaceTB/SmolLM2-135M"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.pad_token = tokenizer.eos_token


In [4]:
model_config = ModelConfig(
    vocab_size=tokenizer.vocab_size,
    d_model=576,
    d_head=64,
    d_mlp_proj=1536,
    n_layers=30,
    n_kv_heads=3,
    n_attn_heads=9,
    rms_norm_eps=1e-5,
    initializer_range=0.041666666666666664,
    rope_theta=100000.0,
    padding_idx=tokenizer.pad_token_id
)

In [5]:
train_config = TrainerConfig(
    per_device_train_batch_size=8,
    max_seq_len=2048,
    num_epochs=50,
    learning_rate=1e-3
)

In [6]:
with open("data/tiny_shakespeare.txt") as f:
    text = f.read()

In [7]:
model = LlamaModel(model_config)
dataloader = DataLoader(train_config, tokenizer, text=text)
trainer = Trainer(train_config, model)

Total train tokens             | 342,016
Num Trainable Params           | 162,826,560
Train device                   | cuda, NVIDIA GeForce RTX 3090, N=1
Training precision             | torch.bfloat16
Flash Attention                | True
torch.compile()                | True





In [ ]:
trainer.train(dataloader)

Training steps                 | 1,050 


In [9]:
input_ids = tokenizer(["HAMLET:\nTo be or"], return_tensors="pt")['input_ids'].to(trainer.device)
idx = model.generate(input_ids, temperature=0.25, top_k=50, max_new_tokens=64)
print(tokenizer.batch_decode(idx)[0])

HAMLET:
To be or power, as I cannot pardon,
And not my lord.

LEONTES:
Farewell, my lord!

KING RICHARD III:
Sweet I love the like to be contrary:
But sanctuary after it was the drunk murd of marriage
As I have strengthen for my
